In [1]:
import os,sys
import pickle
import numpy as np                                       # fast vectors and matrices
from glob import glob
from time import time
import tensorflow as tf
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import shutil

'''
import matplotlib.pyplot as plt                          # plotting
from scipy.fftpack import fft
from sklearn.metrics import average_precision_score
'''
%matplotlib inline

In [2]:
sys.path.insert(0,'library\\')
import config
import diagnostics
import base_model

Instructions for updating:
non-resource variables are not supported in the long term


## data lookup

In [3]:
train_feat_path="processed_dataset\\train\\feat\\"
train_label_path="processed_dataset\\train\\label\\"
test_feat_path="processed_dataset\\test\\feat\\"
test_label_path="processed_dataset\\test\\label\\"

train_name=os.listdir(train_feat_path)
test_name=os.listdir(test_feat_path)

In [4]:
class CustomTrainDataset(Dataset):
    def __init__(self, feat_path=train_feat_path , label_path=train_label_path , transform=None, target_transform=None):
        self.img_labels = os.listdir(train_feat_path)
        self.feat_path = feat_path 
        self.label_path = label_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        label = (torch.from_numpy(np.load(self.label_path +self.img_labels[idx])))
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

class CustomTestDataset(Dataset):
    def __init__(self, feat_path=test_feat_path , label_path=test_label_path , transform=None, target_transform=None):
        self.img_labels = os.listdir(train_feat_path)
        self.feat_path = feat_path 
        self.label_path = label_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        label = (torch.from_numpy(np.load(self.label_path +self.img_labels[idx])))
        '''
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        '''
        print(self.feat_path +self.img_labels[idx])
        return data, label
    
    def demo() :
        for indices in train :
            yield collate_fn(train_dataset[i] for i in indices)


In [5]:
train_dataset=CustomTrainDataset()
test_dataset=CustomTrainDataset()

In [6]:
from torch.utils.data import DataLoader

train = DataLoader(train_dataset, batch_size=32, shuffle=True)
test = DataLoader(train_dataset, batch_size=32, shuffle=True)

## not touch

data : list of list [1500] <br>
data[i] = wav after pass the filter bank in scale [0,120]

label : list of list [1500] <br>
label[i] = 0/1 of if note in index (0-88) is playing


In [ ]:
x=np.arange(0,len(label))
for i in range(len(label)):
    for j in range(88) :
        if(label[i][j]==1) :
            plt.scatter(i,j, s=2,c="b") 
plt.show()

In [ ]:
x=np.arange(0,len(label))
for i in range(len(label)):
    #plt.plot([pt[i] for pt in label])
    for j in range(88) :
        if(label[i][j]==1) :
            plt.scatter(i,j, s=2,c="b") 
plt.show()

## spectrogram

In [7]:
class Spectrograms(base_model.Model):
#class Spectrograms():
    def __init__(self, *args, **kwargs):
        super(Spectrograms, self).__init__(*args, **kwargs)

    def define_graph(self):
        super(Spectrograms, self).define_graph()
        
        # lvl1 convolutions are shared between regions
        self.k = 512              # lvl1 nodes
        self.d = 4096              # lvl1 receptive field
        
        self.k2 = 512              # lvl2 nodes

        # number of lvl1 features
        regions = 1 + (self.window - self.d)/self.stride
        print('Number of V1 feature regions: {}'.format(regions))

        wscale = 10e-3
        with tf.compat.v1.variable_scope('parameters'):
            w = tf.Variable(wscale*tf.compat.v1.random_normal([1,self.d,1,self.k],seed=999))
            wavg = self.register_weights(w,'w',average=.9998)
            w1 = tf.Variable(wscale*tf.compat.v1.random_normal([(int)(regions*self.k),self.k2],seed=999))
            w1avg = self.register_weights(w1,'w1',average=.9998)
            beta = tf.Variable(wscale*tf.compat.v1.random_normal([self.k2,self.m],seed=999))
            betaavg = self.register_weights(beta,'beta',average=.9998)

        with tf.compat.v1.variable_scope('queued_model'):
            zx = tf.compat.v1.log(tf.constant(1.) + tf.nn.relu(tf.nn.conv2d(self.xq,w,strides=[1,1,self.stride,1],padding='VALID')))
            z2 = tf.nn.relu(tf.matmul(tf.reshape(zx,[int(self.batch_size),(int)(regions*self.k)]),w1))
            y = tf.matmul(z2,beta)
            self.loss = tf.reduce_mean(tf.nn.l2_loss(y-tf.reshape(self.yq,[self.batch_size,self.m])))

        with tf.compat.v1.variable_scope('direct_model'):
            zx = tf.compat.v1.log(tf.constant(1.) + tf.nn.relu(tf.nn.conv2d(self.xd,wavg,strides=[1,1,self.stride,1],padding='VALID')))
            z2 = tf.nn.relu(tf.matmul(tf.reshape(zx,[tf.shape(self.xd)[0],int(regions*self.k)]),w1avg))
            self.y_direct = tf.matmul(z2,betaavg)
            self.loss_direct = tf.reduce_mean(tf.nn.l2_loss(self.y_direct-self.yd))

In [8]:
import pandas as pd
df=pd.DataFrame()

model = Spectrograms(train,checkpoint_path='convnet_mlp_raw/', outputs=1, window=16384, mmap=True, batch_size=32,
                    normalize=True, extended_test_set=True, use_mirex=False, init=False, pitch_transforms=5, jitter=.1,
                    restrict=False)
'''
model = Spectrograms(df,checkpoint_path='convnet_mlp_raw/', outputs=1, window=16384, mmap=True, batch_size=200,
                    normalize=True, extended_test_set=True, use_mirex=False, init=False, pitch_transforms=5, jitter=.1,
                    restrict=False)
'''

AttributeError: 'DataLoader' object has no attribute 'keys'

In [6]:
#using dataframe แทน list ที่มี key [อาจจะเป็น dict มั้ง]
import pandas as pd
#df = pd.DataFrame(list(zip(data, label)), columns =['feat', 'label'])
lb={}
lb[1727]=label

In [8]:
temp=os.listdir(config.records_path)

In [5]:
lr = .001/3/3/3
mom = .95

# stop the model if it is already running
#model.stop()

# we have to rebuild the graph every time because input queues can't be reopened
model.define_graph()

learning_rate = tf.compat.v1.placeholder(tf.float32, shape=[])
opt_op = tf.compat.v1.train.MomentumOptimizer(learning_rate,mom).minimize(model.loss)
with tf.control_dependencies([opt_op]):
    train_step = tf.group(*model.averages)

# start up the session, kick off the worker threads, restore checkpoint, etc.
model.start()

try:
    ptime = time()
    print(model.status_header())
    while True:
        if model.iter % 1000 == 0:
            model.update_status(ptime,time(),lr)
            model.checkpoint()
            print(model.status())
            ptime = time()

        model.sess.run(train_step, feed_dict={learning_rate: lr})
        model.iter += 1

except KeyboardInterrupt:
    model.checkpoint()
    print('Graceful Exit')
finally:
    model.stop()

Number of V1 feature regions: 25.0


OSError: [Errno 24] Too many open files: 'processed_dataset/feat/1742-36.npy'